In [1]:
from __future__ import division
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sci_op
# from WindPy import *
from functools import reduce
import datetime

In [2]:
from collections import defaultdict
import pymc3 as pm
import scipy.stats as st

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
plt.style.use('seaborn-darkgrid')
print('Running with PyMC3 version v.{}'.format(pm.__version__))

Running with PyMC3 version v.3.4.1


### getdata

In [4]:
### 511880货基；511010债基
funds=["510330.SH","510880.SH","510900.SH","511880.SH","511010.SH","518880.SH"]
risk_funds=["510330.SH","510880.SH","510900.SH","518880.SH"]

In [5]:
def concat_data(columns):
    data_returns={}
    frame=[]
    for item in data.keys():
        data_returns[item]=data[item].loc[:,columns]
        data_returns[item].rename(columns={columns[1]:item,'日期':'date'},inplace=True)
        frame.append(data_returns[item])
    data1 = reduce(lambda left,right: pd.merge(left,right,on='date'), frame)
    data1.set_index('date', inplace=True)
    data1.dropna(inplace=True)
    return data1

In [6]:
data={}
for item in funds:
    name=item.split('.')[0]
    data[name]=pd.read_excel(name+'.xlsx')

In [7]:
### 收益率
data1=concat_data(['日期','收盘价(元)'])
rets = np.log(data1 / data1.shift(1))
rets.dropna(inplace=True)

In [8]:
n_risk=pd.read_excel('n_risk.xlsx',names=['date','interest'])
n_risk['interest']=n_risk['interest']/100
n_risk.dropna(inplace=True)
n_risk.set_index('date', inplace=True)

### bayes_data

In [15]:
def get_data():
    mu_=np.zeros(6)
    cov_=100*np.identity(6)
    nu_=5
    data=np.array(tmp_rets)
    data = np.random.randn(6,6)
    with pm.Model() as model:
        mu = pm.MvNormal('mu', mu=mu_, cov=cov_,shape=6)
        packed_L = pm.LKJCholeskyCov('packed_L', n=6,eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
        L = pm.expand_packed_triangular(6, packed_L)
        Σ = pm.Deterministic('Σ', L.dot(L.T))
        n = pm.MvNormal('n', mu=mu,chol=L,observed=data)
        trace = pm.sample(5000)

    ppc = pm.sample_ppc(trace, samples=5000, model=model, size=20000)
    r_matrix=np.asarray(ppc['n'])
    return r_matrix
    

### 求解函数

In [16]:
def fun_utility_max(weights):
        miu_=np.mean(sub_rets)
        utility=np.sum(np.sum((weights*sub_rets),axis=1)-riskaverse*(np.sum(weights*(sub_rets-miu_),axis=1))**2)/number
        return -utility


def get_weights():
    averge_weights=np.zeros((5000,6))
    for i in range(0,5000):
        global sub_rets
        sub_rets=mc_rets[i]
        constraints = ({'type':'eq', 'fun':lambda x: np.sum(x)-1.},)
        bounds = tuple((0,1) for x in range(6))
        weights = np.random.random(6)
        weights /= np.sum(weights)
        initial_guess = weights
        opts = sci_op.minimize(fun=fun_utility_max,
                               x0=initial_guess,
                               method='SLSQP',
                               bounds=bounds,
                               constraints=constraints)
        averge_weights[i]=opts['x']
    return np.mean(averge_weights,axis=0)

### 回测过程

In [11]:
def max_drawdown(timeseries):
    # 回撤结束时间点
    i = np.argmax(np.maximum.accumulate(timeseries) - timeseries)
    # 回撤开始的时间点
    j = np.argmax(timeseries[:i])
    return (float(timeseries[i]) / timeseries[j]) - 1.

In [17]:
date_list=rets.index.tolist()
# real_weights={}
cost=0.0003
number=1000
risks=[0.5,1,2]

value=1

with open('bayes_result1_60.csv','w') as f:
    riskaverse=0.5
    for k in range(591,len(date_list),126):
        tmp_rets=rets.iloc[k-252:k,:]
        mc_rets=get_data()
        result=np.array(get_weights())

        for i in range(k,k+126):
            result_list=[]
            day=date_list[i]
            dayoftest=day.strftime('%Y/%m/%d')
            j=i-591
            if j==0:
                change_weight=float(np.sum(abs(result-[0,0,0,0,0,0])))

                rets_change1=float(change_weight*(1-cost)+1-change_weight)
                rets_change2=float(np.sum(result*rets.loc[day,:]))

                real_rets=float(rets_change1*rets_change2+rets_change1-1)
                value=value*(1+real_rets)
            else:
                real_rets=float(np.sum(result*rets.loc[day,:]))
                value=value*(1+real_rets)

            result_list.append(str(dayoftest))
            result_list.extend([str(item) for item in result])
            result_list.append(str(1))
            tmp=[str(item) for item in rets.loc[day,:].tolist()]
            result_list.extend(tmp)
            result_list.append(str(value))
            print(dayoftest,value)
            f.write(','.join(result_list)+'\n')


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [packed_L_cholesky_cov_packed__, mu]
100%|██████████| 5500/5500 [01:16<00:00, 71.77it/s]
There were 41 divergences after tuning. Increase `target_accept` or reparameterize.
There were 125 divergences after tuning. Increase `target_accept` or reparameterize.
There were 52 divergences after tuning. Increase `target_accept` or reparameterize.
There were 47 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.
100%|██████████| 5000/5000 [00:27<00:00, 181.95it/s]


2015/12/31 0.9979563538912521
2016/01/04 0.9708711696039029
2016/01/05 0.9678866196776453
2016/01/06 0.9732615814439908
2016/01/07 0.9507460645994162
2016/01/08 0.9553666082380093
2016/01/11 0.9303963472432376
2016/01/12 0.9213416674523403
2016/01/13 0.9195754586251864
2016/01/14 0.9229077650675309
2016/01/15 0.902584728491921
2016/01/18 0.8998017473452783
2016/01/19 0.9126510462715827
2016/01/20 0.8974909033818556
2016/01/21 0.8850212316212412
2016/01/22 0.896673077094639
2016/01/25 0.9003673687175737
2016/01/26 0.8764769992218063
2016/01/27 0.8788937519200524
2016/01/28 0.8720166810284316
2016/01/29 0.8879702000131088
2016/02/01 0.8846816081157962
2016/02/02 0.8897720890766618
2016/02/03 0.8789794225488082
2016/02/04 0.8900951680029816
2016/02/05 0.8926844350073669
2016/02/15 0.8901449645687931
2016/02/16 0.8999931000413879
2016/02/17 0.8991903630501259
2016/02/18 0.9040022795593141
2016/02/19 0.9058097766534633
2016/02/22 0.9139879342219717
2016/02/23 0.9112168704437023
2016/02/24 0

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [packed_L_cholesky_cov_packed__, mu]
100%|██████████| 5500/5500 [00:55<00:00, 99.63it/s]
There were 20 divergences after tuning. Increase `target_accept` or reparameterize.
There were 38 divergences after tuning. Increase `target_accept` or reparameterize.
There were 58 divergences after tuning. Increase `target_accept` or reparameterize.
There were 50 divergences after tuning. Increase `target_accept` or reparameterize.
100%|██████████| 5000/5000 [00:25<00:00, 194.53it/s]


2016/07/08 0.983227179857328
2016/07/11 0.9902152600128155
2016/07/12 0.9964876295979678
2016/07/13 0.9984973518642303
2016/07/14 1.0001153080495349
2016/07/15 1.0032087845826354
2016/07/18 1.0031594527135395
2016/07/19 1.0004354509031324
2016/07/20 1.0006600009860267
2016/07/21 1.0016890312296787
2016/07/22 0.9983325505909818
2016/07/25 0.9973221169388381
2016/07/26 1.004213613599827
2016/07/27 0.9995368998830751
2016/07/28 1.001498883003661
2016/07/29 0.9994565862707068
2016/08/01 1.0009491061782974
2016/08/02 1.0005342032471247
2016/08/03 1.0009638191780477
2016/08/04 0.9981647609375407
2016/08/05 1.0044156487827354
2016/08/08 1.0072216914571235
2016/08/09 1.0097093260075514
2016/08/10 1.01084190524988
2016/08/11 1.0137029435049802
2016/08/12 1.0209586755021516
2016/08/15 1.0300266746895237
2016/08/16 1.0302791256010897
2016/08/17 1.0285860787894767
2016/08/18 1.0269878587024763
2016/08/19 1.026824967684367
2016/08/22 1.0232312958287555
2016/08/23 1.0233030489260009
2016/08/24 1.020

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [packed_L_cholesky_cov_packed__, mu]
100%|██████████| 5500/5500 [00:54<00:00, 100.15it/s]
There were 26 divergences after tuning. Increase `target_accept` or reparameterize.
There were 17 divergences after tuning. Increase `target_accept` or reparameterize.
There were 30 divergences after tuning. Increase `target_accept` or reparameterize.
There were 38 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.
100%|██████████| 5000/5000 [00:27<00:00, 180.57it/s]


...

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [packed_L_cholesky_cov_packed__, mu]
100%|██████████| 5500/5500 [00:54<00:00, 100.91it/s]
There were 19 divergences after tuning. Increase `target_accept` or reparameterize.
There were 44 divergences after tuning. Increase `target_accept` or reparameterize.
There were 69 divergences after tuning. Increase `target_accept` or reparameterize.
There were 250 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.
100%|██████████| 5000/5000 [00:24<00:00, 206.39it/s]


2017/07/21 1.1097824641429974
2017/07/24 1.1112515938310863
2017/07/25 1.1096570574362625
2017/07/26 1.1073873750189407
2017/07/27 1.109400012356286
2017/07/28 1.109422757795712
2017/07/31 1.1105806061523353
2017/08/01 1.1138107495902008
2017/08/02 1.1135175627692275
2017/08/03 1.1104172372572285
2017/08/04 1.1102837014813334
2017/08/07 1.1110483976465184
2017/08/08 1.1097319930960021
2017/08/09 1.1095652574942632
2017/08/10 1.1088599256696998
2017/08/11 1.1045262829267306
2017/08/14 1.1073382890025845
2017/08/15 1.109101082171791
2017/08/16 1.1071273060997322
2017/08/17 1.1100948159062025
2017/08/18 1.1108471135679014
2017/08/21 1.1110949073111973
2017/08/22 1.1121669611421099
2017/08/23 1.1131982932945474
2017/08/24 1.1124790736391486
2017/08/25 1.1180488666069803
2017/08/28 1.1203801851507875
2017/08/29 1.1209203005228139
2017/08/30 1.1193922393214644
2017/08/31 1.1182788026907924
2017/09/01 1.1197053711040164
2017/09/04 1.1205460790406654
2017/09/05 1.1210498564402356
2017/09/06 1.

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [packed_L_cholesky_cov_packed__, mu]
100%|██████████| 5500/5500 [00:50<00:00, 108.48it/s]
There were 21 divergences after tuning. Increase `target_accept` or reparameterize.
There were 67 divergences after tuning. Increase `target_accept` or reparameterize.
There were 22 divergences after tuning. Increase `target_accept` or reparameterize.
There were 20 divergences after tuning. Increase `target_accept` or reparameterize.
100%|██████████| 5000/5000 [00:24<00:00, 205.47it/s]


2018/01/24 1.1646413681071295
2018/01/25 1.1603061334896838
2018/01/26 1.1674101757150979
2018/01/29 1.1625396435504647
2018/01/30 1.1573667953441409
2018/01/31 1.1567842168810079
2018/02/01 1.1549345164846103
2018/02/02 1.1591028354058557
2018/02/05 1.1623677269927994
2018/02/06 1.1461163996920074
2018/02/07 1.136123640443643
2018/02/08 1.1304934953679846
2018/02/09 1.1153504618153922
2018/02/12 1.1194071003151767
2018/02/13 1.124067987644678
2018/02/14 1.1272485575693818
2018/02/22 1.1353030345160164
2018/02/23 1.1387739580908345
2018/02/26 1.1412302355461188
2018/02/27 1.135303259558285
2018/02/28 1.1301995844821813
2018/03/01 1.1316155290112884
2018/03/02 1.1303366805659436
2018/03/05 1.1292992722313167
2018/03/06 1.134221456996747
2018/03/07 1.1314020806210858
2018/03/08 1.1350717820742233
2018/03/09 1.1363011984515625
2018/03/12 1.1393021065928202
2018/03/13 1.1374010974450288
2018/03/14 1.1352730725309286
2018/03/15 1.1368042836138088
2018/03/16 1.1340315564202068
2018/03/19 1.1

IndexError: list index out of range

In [27]:
result

array([0.02174884, 0.31080452, 0.56820446, 0.02079257, 0.0414816 ,
       0.036968  ])